In [1]:
# One Pickle for NSE

# STATUS: Completed
# Run-time: 1 hour 10 mins

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=1)

In [2]:
%%time
import pandas as pd
import numpy as np
import itertools
import datetime
from math import sqrt, exp, log, erf
import os

#... assignments
exchange = 'NSE'
fspath = './zdata/'


putsigma = 1.5
callsigma = 2
maxdte = 65  # max expiry date for options
mindte = 20  # min expiry date for options

tradingdays = 252

blks = 50

#... Get risk-free rate from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
rate = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

#... Functions
#_____________

#... Error catching for list comprehension

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
        
#...function to get historical data
def get_hist(contract, duration):
    '''Gets 1-day bars of contracts for the duration specified
    Args:
        (contract) as obj
        (duration) as int
    Returns: dataframe of symbol, date, ohlc, avg and volume 
    '''
    
    # Prepare the duration
    strduration = str(duration) + ' D'
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                    durationStr=strduration, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    
    df = util.df(hist)
    df.insert(0, column='symbol', value=contract.symbol)
    
    return df

#...function to get price and dividend ticker
def get_dividend_ticker(contract):
    '''Gets dividend ticker of the contract
    Arg: (contract) as a qualified contract object with conId
    Returns: ticker'''
    
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream

    ticker = ib.ticker(contract)
    
    # Ensure the ticker is filled
    while ticker.dividends is None:
        while np.isnan(ticker.marketPrice()):
            ib.sleep(1)

    ib.cancelMktData(contract)
       
    return ticker

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatility, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatility)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatility
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatility**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1

    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format yyyymmdd
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

#... build the symbols
#______________________

# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct', 'Mlot': 'lot'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_paisa
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_paisa.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

qcs = ib.qualifyContracts(*cs) # qualified underlyings

#****           Single scrip check. To be DELETED in function          *****
#...........................................................................
# contract = next(q for q in qcs if q.symbol=='AMARAJABA')  # one symbol logic check
#___________________________________________________________________________


def get_pkl(contract):

    #... get ohlc, with cumulative volatality and standard deviation
    #_______________________________________________________________

    df_ohlc = get_hist(contract, 365).set_index('date').sort_index(ascending = False)

    # get cumulative standard deviation
    df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
    df_stdev.columns = ['stdev']

    # get cumulative volatility
    df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
    df_vol.columns = ['volatility']

    df_ohlc1 = df_ohlc.join(df_vol)

    df_ohlc2 = df_ohlc1.join(df_stdev)

    #pickle the ohlc
    df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')

    #... get the underlyings
    #_______________________

    ticker = get_dividend_ticker(contract)

    df_und = util.df([ticker])

    cols = ['contract', 'time', 'bid', 'bidSize', 'ask', 'askSize', 'last', 'lastSize', 
            'volume', 'open', 'high', 'low', 'close', 'dividends']
    df_und = df_und[cols]

    df_und = df_und.assign(undPrice=np.where(df_und['last'].isnull(), df_und.close, df_und['last']))

    try: 
        divrate = df_und.dividends[0][0]/df_und.dividends[0][0]/df_und.undPrice
    except (TypeError, AttributeError) as e:
        divrate = 0.0

    df_und = df_und.assign(divrate=divrate)

    df_und = df_und.assign(symbol=[c[1].symbol for c in df_und.contract.items()])

    #... get the lot, margin, undPrice and dividend rate
    undlot = df_paisa.loc[df_paisa.ibSymbol == contract.symbol, 'lot'].item()
    
    df_und['lot'] = undlot

    # margin of underlying
    order = Order(action='SELL', totalQuantity=undlot, orderType='MKT')

    # margin = float(ib.whatIfOrder(contract, order).initMarginChange)
    margin = df_paisa.loc[df_paisa.ibSymbol == contract.symbol, 'TotMgnPerLt'].item()

    df_und['margin'] = margin

    df_und.to_pickle(fspath+contract.symbol+'_und.pkl')

    #... get the options
    #___________________

    # symbol
    symbol = contract.symbol

    # rights
    right = ['P', 'C'] 

    # chains
    chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, futFopExchange='', 
                          underlyingConId=contract.conId, underlyingSecType=contract.secType)

    chain = next(c for c in chains if c.exchange == exchange)

    undPrice = df_und.undPrice[0]

    strikes = sorted([strike for strike in chain.strikes])
    # limit the strikes to outside sigma range
    # strikes = sorted([strike for strike in chain.strikes 
    #            if (strike < (undPrice - std*putsigma)) | (strike > (undPrice + std*callsigma))])

    # limit the expirations to between min and max dates
    expirations = sorted([exp for exp in chain.expirations 
                          if mindte < (util.parseIBDatetime(exp)- datetime.datetime.now().date()).days < maxdte])

    rights = ['P', 'C']

    contracts = [Option(symbol, expiration, strike, right, exchange)
            for right in rights
            for expiration in expirations
            for strike in strikes]

    # Eliminate contracts close to the underlying price
    dtes = [(util.parseIBDatetime(e) - datetime.datetime.now().date()).days for e in expirations]
    sdevs = [df_ohlc2.iloc[i].stdev for i in dtes]


    tgts = []
    for c in contracts:
        if c.right == 'P':
            if c.strike < (undPrice - df_ohlc2.iloc[get_dte(c.lastTradeDateOrContractMonth)].stdev * putsigma):
                tgts.append(c)
        else:
            if c.strike > (undPrice + df_ohlc2.iloc[get_dte(c.lastTradeDateOrContractMonth)].stdev * callsigma):
                tgts.append(c)

    [ib.qualifyContracts(*tgts[i: i+blks]) for i in range(0, len(contracts), blks)]

    qc = [c for c in tgts if c.conId != 0]  # qualified option contracts

    df_opt = pd.concat([pd.DataFrame(qc, columns=['option']), util.df(qc)], axis=1)

    df_opt1 = df_opt[['symbol', 'strike', 'lastTradeDateOrContractMonth', 'right', 'multiplier', 'exchange', 'option']]  # remove unnecessary columns

    df_opt1 = df_opt1.rename(columns={'lastTradeDateOrContractMonth': 'expiry'}) # rename expiry column

    df_opt1 = df_opt1.assign(undPrice=df_opt1.symbol.map(df_und.set_index('symbol')['undPrice']))

    df_opt1['dte'] = (df_opt1.expiry.apply(util.parseIBDatetime) - datetime.datetime.now().date()).dt.days

    df_opt1.loc[df_opt1.dte <= 1, 'dte'] = 2 # Make the dte as 2 for 1 day-to-expiry to prevent bsm divide-by-zero error

    # get the standard deviation based on days to expiry
    df_opt1 = df_opt1.assign(stdev=[df_ohlc2.iloc[i].stdev for i in df_opt1.dte])

    # weed out options within threshold of strike and dte
    mask = (((df_opt1.right == 'P') & (df_opt1.strike < (df_opt1.undPrice - df_opt1.stdev * putsigma))) | \
           ((df_opt1.right == 'C') & (df_opt1.strike > (df_opt1.undPrice + df_opt1.stdev * callsigma))))

    df_opt1 = df_opt1.loc[mask, :].reset_index(drop=True)

    # get the volatality based on days to expiry
    df_opt1 = df_opt1.assign(volatility=[df_ohlc2.iloc[i].volatility for i in df_opt1.dte])

    # get the divrate
    divrate = df_und.divrate[0]

    bsms = [get_bsm(undPrice, strike, dte, rate, volatility, divrate) 
            for undPrice, strike, dte, rate, volatility, divrate in 
            zip(itertools.repeat(undPrice), df_opt1.strike, df_opt1.dte, itertools.repeat(rate), df_opt1.volatility, itertools.repeat(divrate))]

    df_bsm = pd.DataFrame(bsms)

    df_opt2 = df_opt1.join(df_bsm)
    df_opt2['bsmPrice'] = np.where(df_opt2.right == 'P', df_opt2.bsmPut, df_opt2.bsmCall)
    df_opt2['pop'] = np.where(df_opt2.right == 'C', 1-df_opt2.bsmDelta, df_opt2.bsmDelta)
    df_opt2 = df_opt2.drop(['bsmCall', 'bsmPut', 'bsmDelta'], axis=1)

    # get the option prices
    cs = list(df_opt2.option)

    # [catch(lambda: ib.reqTickers(c).marketPrice()) for i in range(0, len(cs), 100) for c in cs[i: i+100]]
    tickers = [ib.reqTickers(*cs[i: i+100]) for i in range(0, len(cs), 100)]

    df_opt3 = df_opt2.assign(price=[t.marketPrice() for ts in tickers for t in ts])

    # filter the options whose price is > 0.0
    df_opt4 = df_opt3[df_opt3.price > 0.0]

    # get the margin of options and make return-on-margin
    co = Order(action='SELL', totalQuantity=1, orderType='MKT')
    # margins = [float(ib.whatIfOrder(contract, order).initMarginChange) for contract, order in zip(df_opt4.option, itertools.repeat(co))]

    df_opt4 = df_opt4.assign(margin=margin)

    df_opt4 = df_opt4.assign(rom=df_opt4.price/df_opt4.margin*tradingdays/df_opt4.dte*undlot)

    df_opt4 = df_opt4.sort_values(by='rom', ascending=False)
    
    # high and low for the options
    df_opt4 = df_opt4.assign(hi52 = df_ohlc2[:252].high.max())
    df_opt4 = df_opt4.assign(lo52 = df_ohlc2[:252].low.min())
    df_opt4.loc[df_opt4.right == 'P', 'hi52'] = np.nan
    df_opt4.loc[df_opt4.right == 'C', 'lo52'] = np.nan

    df_opt4.to_pickle(fspath+contract.symbol+'_opt.pkl')
    
    return None

Started to throttle requests
Stopped to throttle requests


Wall time: 11.9 s


In [3]:
%%time

symbols = [s.symbol for s in qcs]

# Pickle the dataframes
fspath = './zdata/'
fs = os.listdir(fspath)

# If the path is empty, start filling it in blocks of 50 underlyings
# Else start from where you left!
if fs == []:
    [catch(lambda: get_pkl(t)) for t in qcs]
#     [get_pkl(t) for i in range(0, len(qcs), 50) for t in qcs[i: i+50]]
    ib.disconnect()
    
else:
    # Take only pickle files. Remove directories and files starting with underscore (for underlyings)
    fs = [f for f in fs if (f[-7:] == 'opt.pkl')]

    # Get modified time, fail time and identify where the scrip has failed
    fsmod = {f: os.path.getmtime(fspath + '/' + f) for f in fs}

    failtime = max([v for k, v in fsmod.items()])

    failscrip = [k[:-4] for k, v in fsmod.items() if v == failtime][0]    

    restartfrom = symbols.index(failscrip[:-4]) + 1

    xs = [f[:-8] for f in fs] # existing symbols

    rc = [c for c in qcs if c.symbol not in xs] # remaining qcs

    # Restart the pickling!
    [catch(lambda: get_pkl(t)) for t in rc]
    
    ib.disconnect()

Error 200, reqId 290: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190328', strike=1920.0, right='C', exchange='NSE')
Error 200, reqId 291: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190328', strike=1940.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190328', strike=1920.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190328', strike=1940.0, right='C', exchange='NSE')
Started to throttle requests
Stopped to throttle requests
Error 200, reqId 516: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTradeDateOrContractMonth='20190328', strike=255.0, right='P', exchange='NSE')
Error 200, reqId 518: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTr

Started to throttle requests
Stopped to throttle requests
Error 200, reqId 1421: No security definition has been found for the request, contract: Option(symbol='APOLLOHOS', lastTradeDateOrContractMonth='20190328', strike=780.0, right='P', exchange='NSE')
Error 200, reqId 1422: No security definition has been found for the request, contract: Option(symbol='APOLLOHOS', lastTradeDateOrContractMonth='20190328', strike=800.0, right='P', exchange='NSE')
Error 200, reqId 1423: No security definition has been found for the request, contract: Option(symbol='APOLLOHOS', lastTradeDateOrContractMonth='20190328', strike=820.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='APOLLOHOS', lastTradeDateOrContractMonth='20190328', strike=780.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='APOLLOHOS', lastTradeDateOrContractMonth='20190328', strike=800.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='APOLLOHOS', lastTradeDateOrContractMonth='20190328', strike=820.

Unknown contract: Option(symbol='ASHOKLEY', lastTradeDateOrContractMonth='20190328', strike=165.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ASHOKLEY', lastTradeDateOrContractMonth='20190328', strike=170.0, right='C', exchange='NSE')
Started to throttle requests
Stopped to throttle requests
Error 200, reqId 2097: No security definition has been found for the request, contract: Option(symbol='ASIANPAIN', lastTradeDateOrContractMonth='20190328', strike=880.0, right='P', exchange='NSE')
Error 200, reqId 2098: No security definition has been found for the request, contract: Option(symbol='ASIANPAIN', lastTradeDateOrContractMonth='20190328', strike=900.0, right='P', exchange='NSE')
Error 200, reqId 2099: No security definition has been found for the request, contract: Option(symbol='ASIANPAIN', lastTradeDateOrContractMonth='20190328', strike=920.0, right='P', exchange='NSE')
Error 200, reqId 2100: No security definition has been found for the request, contract: Option(symb

Unknown contract: Option(symbol='BAJAJ-AUT', lastTradeDateOrContractMonth='20190328', strike=3850.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJ-AUT', lastTradeDateOrContractMonth='20190328', strike=3900.0, right='C', exchange='NSE')
Error 200, reqId 2757: No security definition has been found for the request, contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=3700.0, right='P', exchange='NSE')
Error 200, reqId 2759: No security definition has been found for the request, contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=3800.0, right='P', exchange='NSE')
Error 200, reqId 2760: No security definition has been found for the request, contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=3900.0, right='P', exchange='NSE')
Error 200, reqId 2762: No security definition has been found for the request, contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190

Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=5900.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=6800.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=6900.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=7100.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=7200.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=7300.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=7400.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BAJAJFINS', lastTradeDateOrContractMonth='20190328', strike=7600.0, rig

...

Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190228', strike=8500.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190228', strike=8600.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190228', strike=8700.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190228', strike=8800.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190228', strike=8900.0, right='P', exchange='NSE')
Error 200, reqId 30872: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=4600.0, right='P', exchange='NSE')
Error 200, reqId 30873: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=4700.0, rig

Error 200, reqId 30900: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=7400.0, right='P', exchange='NSE')
Error 200, reqId 30901: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=7500.0, right='P', exchange='NSE')
Error 200, reqId 30917: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=9050.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=6300.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=6400.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190328', strike=6500.0, right='P', exchange='NSE')
Unknown contract: Option(symbol

Error 200, reqId 31224: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307', strike=24700.0, right='P', exchange='NSE')
Error 200, reqId 31226: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307', strike=24900.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307', strike=24500.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307', strike=24600.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307', strike=24700.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307', strike=24800.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190307'

Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190314', strike=29200.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190314', strike=29300.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190320', strike=29200.0, right='C', exchange='NSE')
Error 200, reqId 31397: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190320', strike=29300.0, right='C', exchange='NSE')
Error 200, reqId 31418: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190404', strike=29100.0, right='C', exchange='NSE')
Error 200, reqId 31419: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190404', strike=29200.0, right='C', exchange='NSE')
Error 200, re

Wall time: 1h 18min 12s


In [ ]:
ib.disconnect()
ib.sleep(4)
ib = IB().connect('127.0.0.1', 3000, clientId=1)